In [14]:
# import Python packages
import pandas as pd
import numpy as np
from numpy import array
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.gridspec import GridSpec
import scipy
import scipy.stats as ss
from skbio.stats.distance import permanova
import biom
from biom import load_table
from gemelli.rpca import rpca #import auto_rpca
from gemelli.utils import qc_rarefaction
from skbio.stats.distance import permanova, DistanceMatrix
from adjustText import adjust_text
### See RPCA standalone python tutorial: 
# https://github.com/biocore/gemelli/blob/master/ipynb/tutorials/RPCA-moving-pictures-standalone-cli-and-api.ipynb

In [15]:
# read in biom table
biom_tbl = biom.load_table("metaphlan4species_readstats_transposed.biom")

# perform RPCA with auto rank estimation
np.seterr(divide = 'ignore')
#ordination, distance = auto_rpca(biom_tbl)
ordination, distance = rpca(biom_tbl)#, min_sample_count=6103)

In [16]:
print(ordination.proportion_explained)

PC1    0.513280
PC2    0.296326
PC3    0.190394
dtype: float64


In [17]:
#We can see here that the difference between the rarefy distance vs. unrarefy distance in the correlation to the absolute difference in sample sums is non-significant. If it was significant you would need to continue on using only the rarefy results but since it is not here we will continue with the unrarefy.
rare_ordination, rare_distance = rpca(biom_tbl.copy().subsample(1603))
t, p = qc_rarefaction(biom_tbl, rare_distance, distance)
(t, p)
#non-significant results indicate that we can continue with the non-rarefied data

(-0.7237810718350511, 0.4703743093892938)

In [18]:
# Read the metadata file
metadata_file = 'metadata_metaphlan.csv'
metadata_df = pd.read_csv(metadata_file)

metadata_df.head()
print(metadata_df.columns)
print(metadata_df)

Index(['Sample', 'subjectID', 'Sample_name', 'Body_site', 'Body_site_2',
       'Study', 'Pool', 'Library_Concentration_ng_uL', 'Age', 'Sex',
       'Weight_kg', 'Length_cm', 'BMI', 'DNAExtraction_method'],
      dtype='object')
      Sample subjectID                Sample_name     Body_site   Body_site_2  \
0    X16B1NL    MAN-16  X16B1NL_CP04272_S343_L001  Non_Lesional  Non_Lesional   
1     X17B1L    MAN-17   X17B1L_CP04272_S344_L001      Lesional      Lesional   
2    X17B1NL    MAN-17  X17B1NL_CP04272_S273_L001  Non_Lesional  Non_Lesional   
3      X2B1L     MAN-2    X2B1L_CP04272_S276_L001      Lesional      Lesional   
4     X2B1NL     MAN-2   X2B1NL_CP04272_S277_L001  Non_Lesional  Non_Lesional   
..       ...       ...                        ...           ...           ...   
142    UG004       UG2    UG004_CP04272_S260_L001    Back_scalp       Healthy   
143    UG005       UG3    UG005_CP04272_S261_L001     Top_scalp       Healthy   
144    UG006       UG3    UG006_CP04272_S2

In [19]:
# extract and view sample ordinations from RPCA result
spca_df = ordination.samples

# convert data to array
spca_df_arr = array(spca_df)
spca_df.head()

,PC1,PC2,PC3
UG010,-0.031170,-0.010884,-0.121625
UG009,0.020536,0.056773,-0.022677
UG006,0.021479,0.001800,-0.047204
UG005,-0.061922,0.123513,0.027721
UG004,0.067476,0.082804,-0.025955


In [20]:
# view distance matrix
print(distance)

147x147 distance matrix
IDs:
'UG010', 'UG009', 'UG006', 'UG005', 'UG004', 'UG003', 'UG002', 'UG001', ...
Data:
[[0.         1.56408643 1.01139717 ... 2.19566163 0.53743168 4.18009715]
 [1.56408643 0.         0.70882604 ... 2.90938861 1.9040235  3.47457203]
 [1.01139717 0.70882604 0.         ... 2.25206125 1.24083419 3.37552338]
 ...
 [2.19566163 2.90938861 2.25206125 ... 0.         1.75323013 3.13443236]
 [0.53743168 1.9040235  1.24083419 ... 1.75323013 0.         4.03435055]
 [4.18009715 3.47457203 3.37552338 ... 3.13443236 4.03435055 0.        ]]


In [21]:
# Verify the columns in metadata_df
print("Metadata columns:", metadata_df.columns)

# Verify the first few rows of spca_df and metadata_df to ensure the data is as expected
print("SPCA DataFrame (first few rows):\n", spca_df.head())
print("Metadata DataFrame (first few rows):\n", metadata_df.head())

# Check if the index of spca_df is correctly read and contains sample names
print("Index of spca_df (Sample Names):\n", spca_df.index)

Metadata columns: Index(['Sample', 'subjectID', 'Sample_name', 'Body_site', 'Body_site_2',
       'Study', 'Pool', 'Library_Concentration_ng_uL', 'Age', 'Sex',
       'Weight_kg', 'Length_cm', 'BMI', 'DNAExtraction_method'],
      dtype='object')
SPCA DataFrame (first few rows):
             PC1       PC2       PC3
UG010 -0.031170 -0.010884 -0.121625
UG009  0.020536  0.056773 -0.022677
UG006  0.021479  0.001800 -0.047204
UG005 -0.061922  0.123513  0.027721
UG004  0.067476  0.082804 -0.025955
Metadata DataFrame (first few rows):
     Sample subjectID                Sample_name     Body_site   Body_site_2  \
0  X16B1NL    MAN-16  X16B1NL_CP04272_S343_L001  Non_Lesional  Non_Lesional   
1   X17B1L    MAN-17   X17B1L_CP04272_S344_L001      Lesional      Lesional   
2  X17B1NL    MAN-17  X17B1NL_CP04272_S273_L001  Non_Lesional  Non_Lesional   
3    X2B1L     MAN-2    X2B1L_CP04272_S276_L001      Lesional      Lesional   
4   X2B1NL     MAN-2   X2B1NL_CP04272_S277_L001  Non_Lesional  Non_Les

In [22]:
# Map the index of spca_df to the Sample column of metadata_df
# Assuming the index of spca_df contains the sample names
spca_df["Body_site"] = spca_df.index.map(metadata_df.set_index("Sample")["Body_site"])

# Display the updated spca_df
print(spca_df)

              PC1       PC2       PC3     Body_site
UG010   -0.031170 -0.010884 -0.121625    Back_scalp
UG009    0.020536  0.056773 -0.022677     Top_scalp
UG006    0.021479  0.001800 -0.047204    Back_scalp
UG005   -0.061922  0.123513  0.027721     Top_scalp
UG004    0.067476  0.082804 -0.025955    Back_scalp
...           ...       ...       ...           ...
X2B1NL   0.118915 -0.010971 -0.043397  Non_Lesional
X2B1L   -0.037552 -0.042450 -0.046768      Lesional
X17B1NL  0.101887 -0.155420 -0.114401  Non_Lesional
X17B1L  -0.018465 -0.071842 -0.105057      Lesional
X16B1NL  0.264492 -0.143390  0.138186  Non_Lesional

[147 rows x 4 columns]


In [23]:
# extract and view feature ordinations from RPCA result
fpca_df = ordination.features
fpca_df.head()

,PC1,PC2,PC3
s__Kocuria_sp_cx_116,-0.000823,0.003413,-0.005335
s__Pyrinomonas_methylaliphatogenes,0.007653,0.003620,0.027451
s__Actinobaculum_sp_oral_taxon_183,-0.001340,-0.001000,-0.004574
s__Actinomyces_SGB17157,0.001488,0.002546,0.003342
s__Actinomyces_SGB17163,-0.005830,0.004977,0.002065


In [24]:
# create color palette
colors = ["#018571", "#80cdc1", "#a6611a","#dfc27d"]
my_colors = sns.set_palette(sns.color_palette(colors))

Body_site =["Top_scalp", "Back_scalp", "Lesional", "Non_Lesional"]
palette = dict(zip(
    Body_site,
    sns.color_palette(my_colors, len(Body_site))

))

In [25]:
# calculate permanova F-statistic
pnova_res = permanova(distance, spca_df, "Body_site")
print(pnova_res)

method name               PERMANOVA
test statistic name        pseudo-F
sample size                     147
number of groups                  4
test statistic             4.068151
p-value                       0.001
number of permutations          999
Name: PERMANOVA results, dtype: object


In [26]:
# extract and view sample ordinations from RPCA result
#spca_df = ordination.samples


# convert data to array
spca_df_arr = array(spca_df)
spca_df.head()

# Ensure both spca_df and metadata_df are loaded correctly
# Convert the index of spca_df to string if it's not already
spca_df.index = spca_df.index.astype(str)

# Ensure the 'Sample' column in metadata_df is also of type string
metadata_df["Sample"] = metadata_df["Sample"].astype(str)

# Verify the columns in metadata_df
print("Metadata columns:", metadata_df.columns)

# Verify the first few rows of spca_df and metadata_df to ensure the data is as expected
print("SPCA DataFrame (first few rows):\n", spca_df.head())
print("Metadata DataFrame (first few rows):\n", metadata_df.head())

# Check if the index of spca_df is correctly read and contains sample names
print("Index of spca_df (Sample Names):\n", spca_df.index)

Metadata columns: Index(['Sample', 'subjectID', 'Sample_name', 'Body_site', 'Body_site_2',
       'Study', 'Pool', 'Library_Concentration_ng_uL', 'Age', 'Sex',
       'Weight_kg', 'Length_cm', 'BMI', 'DNAExtraction_method'],
      dtype='object')
SPCA DataFrame (first few rows):
             PC1       PC2       PC3   Body_site
UG010 -0.031170 -0.010884 -0.121625  Back_scalp
UG009  0.020536  0.056773 -0.022677   Top_scalp
UG006  0.021479  0.001800 -0.047204  Back_scalp
UG005 -0.061922  0.123513  0.027721   Top_scalp
UG004  0.067476  0.082804 -0.025955  Back_scalp
Metadata DataFrame (first few rows):
     Sample subjectID                Sample_name     Body_site   Body_site_2  \
0  X16B1NL    MAN-16  X16B1NL_CP04272_S343_L001  Non_Lesional  Non_Lesional   
1   X17B1L    MAN-17   X17B1L_CP04272_S344_L001      Lesional      Lesional   
2  X17B1NL    MAN-17  X17B1NL_CP04272_S273_L001  Non_Lesional  Non_Lesional   
3    X2B1L     MAN-2    X2B1L_CP04272_S276_L001      Lesional      Lesional  

In [27]:


# Merge spca_df with metadata_df to include "Body_site" and "Pool"
spca_df = spca_df.merge(metadata_df[["Sample", "Pool"]], left_index=True, right_on="Sample")

# Create a new column that combines Body_site and Pool
spca_df['Body_site_Pool'] = spca_df['Pool'] + " " + spca_df['Body_site']

# Display the updated spca_df
print("SPCA DataFrame with combined column (first few rows):\n", spca_df.head())

# Create a color palette for the new combined column
# Using the same colors as the previous example for the Body_site
colors = ["#018571", "#80cdc1", "#a6611a", "#dfc27d"]
body_site_colors = sns.color_palette(sns.color_palette(colors))

Body_site = ["Top_scalp", "Back_scalp", "Lesional", "Non_Lesional"]
body_site_palette = dict(zip(
    Body_site,
    body_site_colors
))

# Map the combined color palette to the new combined column
combined_colors = {}
for pool in ["Scalp psoriasis", "Healthy"]:
    for site in Body_site:
        combined_colors[f"{pool} {site}"] = body_site_palette[site]

# Map markers to the new combined column
marker_mapping = {
    "Scalp psoriasis": "^",  # Triangle
    "Healthy": "o"  # Circle
}

# Map markers to the new combined column
spca_df['Marker'] = spca_df['Pool'].map(marker_mapping)

# Set plot parameters
mm = 1/25.4
fig, ax = plt.subplots(1, 1, figsize=(90*mm, 110*mm))

fpca_df.columns = [f"PC{i+1}" for i in range(fpca_df.shape[1])]

# Create scatter plot with different markers and colors based on the new combined column
for (body_site_pool, marker), group in spca_df.groupby(['Body_site_Pool', 'Marker']):
    sns.scatterplot(
        data=group,
        x="PC1",
        y="PC2",
        color=combined_colors[body_site_pool],
        marker=marker,
        edgecolor=None,
        ax=ax,
        label=body_site_pool
    )

# Draw convex hulls
for body_site_pool, group in spca_df.groupby('Body_site_Pool'):
    color = combined_colors[body_site_pool]
    points = group[["PC1", "PC2"]].values
    hull = scipy.spatial.ConvexHull(points)
    hull_plot_x = points[hull.vertices, 0]
    hull_plot_y = points[hull.vertices, 1]
    hull_plot_x = np.append(hull_plot_x, points[hull.vertices[0], 0])
    hull_plot_y = np.append(hull_plot_y, points[hull.vertices[0], 1])
    ax.plot(hull_plot_x, hull_plot_y, c=color, zorder=0)
    ax.fill(hull_plot_x, hull_plot_y, c=color, alpha=0.3)

# Calculate the top 10 features based on the sum of absolute loadings on PC1 and PC2
fpca_df['sum_abs_loadings'] = fpca_df[['PC1', 'PC2']].abs().sum(axis=1)
top_features = fpca_df.nlargest(6, 'sum_abs_loadings').index

# Calculate the top features with negative loadings on PC2
top_negative_pc2_features = fpca_df.nsmallest(2, 'PC2').index

# Combine both sets of features
combined_features = list(set(top_features) | set(top_negative_pc2_features))

# Add biplot arrows and labels
scaling_factor = 0.2  # Adjust the scaling factor if necessary
texts = []
for feature in combined_features:  # Combined features
    arrow_x = fpca_df.loc[feature, 'PC1'] * scaling_factor
    arrow_y = fpca_df.loc[feature, 'PC2'] * scaling_factor
    ax.arrow(0, 0, arrow_x, arrow_y,
             color='grey', alpha=0.7, head_width=0.01)
    
    # Adjust label position slightly to the right or left
    if arrow_x > 0:
        label_x = arrow_x  # Place label to the right of the arrow
        ha = 'left'
    else:
        label_x = arrow_x  # Place label to the left of the arrow
        ha = 'right'
    
    # Additional adjustment for specific cases to avoid overlap
    if feature == "s__Cutibacterium_avidum":
        label_y = arrow_y - 0.01  # Place the label slightly below the arrow
        label_x = arrow_x - 0.14
    elif feature == "s__Cutibacterium_porci":
        label_y = arrow_y  # Place the label slightly above the arrow
        label_x = arrow_x + 0.005
    else:
        label_y = arrow_y
    
    texts.append(ax.text(label_x, label_y,
                         feature, color='black', fontsize=5, ha=ha, va='center'))

# Adjust text to avoid overlap
adjust_text(texts)

# Customize legend and labels
handles, labels = ax.get_legend_handles_labels()
plt.legend(
    handles=handles,
    labels=labels,
    frameon=False,
    fontsize=7,
    title_fontsize=7
)

pc1_pct, pc2_pct, _ = [f"PC{i+1} ({x*100:.2f}%)" for i, x in enumerate(ordination.proportion_explained)]

ax.set_xlabel(pc1_pct, fontsize=7)
ax.set_ylabel(pc2_pct, fontsize=7)

yticklabels = [-0.2, -0.1, 0.0, 0.1, 0.2, 0.3]
yticklocations = yticklabels
ax.set_yticks(yticklocations)
ax.set_yticklabels(yticklabels, fontsize=7)

xticklabels = [-0.2, -0.1, 0.0, 0.1, 0.2, 0.3]
xticklocations = xticklabels
ax.set_xticks(xticklocations)
ax.set_xticklabels(xticklabels, fontsize=7)

ax.text(-0.18, 0.27, 'PERMANOVA', fontsize=7)
ax.text(-0.18, 0.25, '***p-val = 0.001', fontsize=7)

ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)

plt.tight_layout()
plt.savefig("RPCA.png", dpi=600)
plt.show()


SPCA DataFrame with combined column (first few rows):
           PC1       PC2       PC3   Body_site Sample     Pool  \
146 -0.031170 -0.010884 -0.121625  Back_scalp  UG010  Healthy   
145  0.020536  0.056773 -0.022677   Top_scalp  UG009  Healthy   
144  0.021479  0.001800 -0.047204  Back_scalp  UG006  Healthy   
143 -0.061922  0.123513  0.027721   Top_scalp  UG005  Healthy   
142  0.067476  0.082804 -0.025955  Back_scalp  UG004  Healthy   

         Body_site_Pool  
146  Healthy Back_scalp  
145   Healthy Top_scalp  
144  Healthy Back_scalp  
143   Healthy Top_scalp  
142  Healthy Back_scalp  


/var/folders/g4/bhmr4hbn3pn5zc7fy0ffk9gh0000gp/T/ipykernel_46582/3316183439.py:143: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
